### Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import ExactMatch

In [ ]:
data_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/data/nq_open/validation-00000-of-00001.parquet"

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet(data_file_path, engine='pyarrow')

In [ ]:
df.head()

,question,answer
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]"
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]"
2,how many seasons of the bastard executioner ar...,"[one, one season]"
3,when did the eagles win last super bowl,[2017]
4,who won last year's ncaa women's basketball,[South Carolina]


### Load LLM

In [ ]:
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY = "************************************************************"

In [ ]:
MODEL_NAME = "gpt-3.5-turbo"

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(
      model=MODEL_NAME,
      temperature=0.1,
      max_tokens=10,
      api_key=OPENAI_API_KEY
    )

In [ ]:
resp = llm.complete("Paul Graham is ")
print(resp)

a computer scientist, entrepreneur, and venture capitalist.


### Inference

In [ ]:
!pip install tqdm

In [ ]:
PROMPT_TEMPLATE = """
[INST] <<SYS>>
This is a Open Domain Question Answering Task. You are given a question and you need to generate
the short answer based on the question. Below are a few examples:
Question: where did they film hot tub time machine
Answer: Fernie Alpine Resort
Question: who has the right of way in international waters
Answer: Neither vessel
Question: who does annie work for attack on titan
Answer: Marley
Question: when was the immigration reform and control act passed
Answer: November 6, 1986
[ENDSYS]
<</SYS>>
Question: {prompt}[/INST]
"""

In [ ]:
def generate_answer(question, index):
    # Format the PROMPT_TEMPLATE with the actual question
    formatted_prompt = PROMPT_TEMPLATE.format(prompt=question)
    try:
        response = llm.complete(formatted_prompt)
        answer = response.text
        print(f"Index: {index}, Question: '{question}', Generated Answer: '{answer}'")
        return answer
    except Exception as e:
        print(f"Error generating answer for question '{question}' at index {index}: {e}")
        return None

In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_gpt-3.5-turbo.csv"

In [ ]:
try:
    # Try to load the DataFrame from the CSV file
    df_eval = pd.read_csv(eval_file_path)
    print("Data loaded from file.")
except FileNotFoundError:
    # If the file doesn't exist, copy df into df_eval
    df_eval = df.copy()
    if 'predictions' not in df_eval.columns:
      df_eval['predictions'] = pd.NA  # or use None or np.nan, depending on your preference
    print("File not found. Copied df into df_eval.")

File not found. Copied df into df_eval.


In [ ]:
from tqdm import tqdm

# Initialize a counter for tracking when to save the DataFrame
save_counter = 0

for index, row in tqdm(df_eval.iterrows(), total=df_eval.shape[0], desc="Generating Answers"):
    # Check if 'predictions' is empty for the current row
    if pd.isna(row['predictions']) or row['predictions'] == '':
        # Generate the answer only if 'predictions' is empty
        df_eval.at[index, 'predictions'] = generate_answer(row['question'], index)

        # Increment the save counter
        save_counter += 1

        # Save the DataFrame back to the file every 10 rows
        if save_counter % 10 == 0:
            df_eval.to_csv(eval_file_path, index=False)
            print(f"Progress saved at index {index}")

# Save any remaining progress at the end of the loop
if save_counter % 10 != 0:
    df_eval.to_csv(eval_file_path, index=False)
    print("Final progress saved.")

Generating Answers:   0%|          | 1/3610 [00:00<34:55,  1.72it/s]

Index: 0, Question: 'when was the last time anyone was on the moon', Generated Answer: 'Answer: The last time anyone was on the moon'


Generating Answers:   0%|          | 2/3610 [00:00<23:26,  2.56it/s]

Index: 1, Question: 'who wrote he ain't heavy he's my brother lyrics', Generated Answer: 'Answer: Bobby Scott and Bob Russell'


Generating Answers:   0%|          | 3/3610 [00:01<25:45,  2.33it/s]

...

Index: 3607, Question: 'who will take the throne after the queen dies', Generated Answer: 'Answer: The next in line for the throne after'


Generating Answers: 100%|█████████▉| 3609/3610 [30:53<00:00,  2.02it/s]

Index: 3608, Question: 'what domain has more individuals than all other domains combined do', Generated Answer: 'Answer: The domain of bacteria has more individuals than'


Generating Answers: 100%|██████████| 3610/3610 [30:54<00:00,  1.95it/s]

Index: 3609, Question: 'what is the meaning of the name comanche', Generated Answer: 'Answer: The name Comanche is believed to mean'
Progress saved at index 3609


In [ ]:
df_eval.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",Answer: The last time anyone was on the moon
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",Answer: Bobby Scott and Bob Russell
2,how many seasons of the bastard executioner ar...,"[one, one season]","Answer: There is only one season of ""The"
3,when did the eagles win last super bowl,[2017],Answer: The Philadelphia Eagles won their last...
4,who won last year's ncaa women's basketball,[South Carolina],Answer: Stanford University


## Save evaluation data

In [ ]:
df_eval.to_csv(eval_file_path, index=False)

## Evaluation metrics

In [ ]:
!pip install pandas pyarrow datasets

In [ ]:
import pandas as pd

In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_gpt-3.5-turbo.csv"

In [ ]:
df = pd.read_csv(eval_file_path)

In [ ]:
df.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,['14 December 1972 UTC' 'December 1972'],Answer: The last time anyone was on the moon
1,who wrote he ain't heavy he's my brother lyrics,['Bobby Scott' 'Bob Russell'],Answer: Bobby Scott and Bob Russell
2,how many seasons of the bastard executioner ar...,['one' 'one season'],"Answer: There is only one season of ""The"
3,when did the eagles win last super bowl,['2017'],Answer: The Philadelphia Eagles won their last...
4,who won last year's ncaa women's basketball,['South Carolina'],Answer: Stanford University


In [ ]:
print(df.iloc[1]['answer'])

['Bobby Scott' 'Bob Russell']


In [ ]:
import ast
import string
import re
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import normalize_answer

def string_to_list(string):
    try:
        # Convert the string to a list
        matches = re.findall(r"'(.*?)'", string)
        return matches
    except Exception as e:
        # In case of an error, print the error and return the original string
        print(f"Error converting string to list: {e}")
        return string


def preprocess(text: str, answers: list) -> str:
    text_after_answer = re.sub(r'^Answer: ', '', text)

    exclude = set(string.punctuation)
    exclude.add("’")
    exclude.add("\n")

    # Remove excluded characters
    cleaned_text = normalize_answer("".join(ch for ch in text_after_answer if ch not in exclude))

    # Replace text with answer item if it matches a substring in the cleaned text
    for answer_item in answers:
        if normalize_answer(answer_item) in cleaned_text:
            return answer_item  # Return the answer item that matches the substring

    return cleaned_text

In [ ]:
df['answer'] = df['answer'].apply(string_to_list)
df['predictions'] = df['predictions'].str.replace("^Answer: ", "", regex=True)
df['predictions_em'] = df.apply(lambda row: preprocess(row['predictions'], row['answer']), axis=1)

In [ ]:
df.head()

,question,answer,predictions,predictions_em
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",The last time anyone was on the moon,last time anyone was on moon
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",Bobby Scott and Bob Russell,Bobby Scott
2,how many seasons of the bastard executioner ar...,"[one, one season]","There is only one season of ""The",one
3,when did the eagles win last super bowl,[2017],The Philadelphia Eagles won their last Super Bowl,philadelphia eagles won their last super bowl
4,who won last year's ncaa women's basketball,[South Carolina],Stanford University,stanford university


### Calculate Scores

In [ ]:
import datasets
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import exact_match_score, metric_max_over_ground_truths, f1_score

df['em'] = df.apply(lambda row: metric_max_over_ground_truths(exact_match_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)
df['f1'] = df.apply(lambda row: metric_max_over_ground_truths(f1_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)

In [ ]:
!pip install -q rouge_score
!pip install -q evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
import evaluate

df['predictions_em'] = df['predictions_em'].astype(str)
df['answer'] = df['answer'].astype(str)

# Filter out rows where either 'predictions' or 'answer' is empty or NaN
filtered_df = df[(df['predictions_em'].notna() & df['predictions_em'].str.strip().astype(bool)) &
                 (df['answer'].notna() & df['answer'].str.strip().astype(bool))]
rouge = evaluate.load('rouge')
predictions = filtered_df['predictions_em'].tolist()
references = filtered_df['answer'].tolist()
rouge_results = rouge.compute(predictions=predictions,
                        references=references)
print(rouge_results)

{'rouge1': 0.3098870451686362, 'rouge2': 0.18065528339787978, 'rougeL': 0.30819172992550214, 'rougeLsum': 0.30803937967511763}


### Prove of correctness

#### Top 10 rows from data frame

In [ ]:
df.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",The last time anyone was on the moon,last time anyone was on moon,False,0.0
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott and Bob Russell,Bobby Scott,True,1.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']","There is only one season of ""The",one,True,1.0
3,when did the eagles win last super bowl,['2017'],The Philadelphia Eagles won their last Super Bowl,philadelphia eagles won their last super bowl,False,0.0
4,who won last year's ncaa women's basketball,['South Carolina'],Stanford University,stanford university,False,0.0
5,when did the isle of wight become an island,['During the last Ice Age'],The Isle of Wight became an island,isle of wight became island,False,0.0
6,love yourself by justin bieber is about who,['Rihanna'],"""Love Yourself"" by Justin Bieber is",love yourself by justin bieber is,False,0.0
7,who was the ruler of england in 1616,['James I'],King James I,James I,True,1.0
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],The Hot Coffee mod in San Andreas is,hot coffee mod in san andreas is,False,0.0
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],54 Mbps,54 mbps,False,0.0


#### Top 10 rows from data frame that exactly match one of the reference answer

In [ ]:
df_em_true = df[df['em'] == True]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",Bobby Scott and Bob Russell,Bobby Scott,True,1.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']","There is only one season of ""The",one,True,1.0
7,who was the ruler of england in 1616,['James I'],King James I,James I,True,1.0
10,which state is located in the centre of india,"['Chhattisgarh', 'Madhya Pradesh']",Madhya Pradesh,Madhya Pradesh,True,1.0
11,who sang i ran all the way home,['The Impalas'],The Impalas,The Impalas,True,1.0
13,who was the actor that played ben stone on law...,['Michael Moriarty'],Michael Moriarty,Michael Moriarty,True,1.0
17,who is under the mask of darth vader,['Anakin Skywalker'],Anakin Skywalker,Anakin Skywalker,True,1.0
18,who had a baby at 100 in the bible,"['Sarah', 'Abraham']","Sarah, the wife of Abraham, had",Sarah,True,1.0
22,how many seasons of vampire diaries r there,"['eight', '8']",There are 8 seasons of Vampire Di,8,True,1.0
23,how many episodes are there in dragon ball z,"['291 episodes', '291']",There are 291 episodes in Dragon Ball,291 episodes,True,1.0


#### Top 10 rows from data frame that doesn't match match any of the reference answer

In [ ]:
df_em_true = df[df['em'] == False]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",The last time anyone was on the moon,last time anyone was on moon,False,0.000000
3,when did the eagles win last super bowl,['2017'],The Philadelphia Eagles won their last Super Bowl,philadelphia eagles won their last super bowl,False,0.000000
4,who won last year's ncaa women's basketball,['South Carolina'],Stanford University,stanford university,False,0.000000
5,when did the isle of wight become an island,['During the last Ice Age'],The Isle of Wight became an island,isle of wight became island,False,0.000000
6,love yourself by justin bieber is about who,['Rihanna'],"""Love Yourself"" by Justin Bieber is",love yourself by justin bieber is,False,0.000000
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],The Hot Coffee mod in San Andreas is,hot coffee mod in san andreas is,False,0.000000
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],54 Mbps,54 mbps,False,0.000000
12,where did the last name wallace come from,['a Scottish surname'],The last name Wallace is of Scottish origin,last name wallace is of scottish origin,False,0.222222
14,who does the voice of nala in the lion king,"['Niketa Calame', 'Sally Dworsky', 'Moira Kell...",Beyoncé,beyoncé,False,0.000000
15,who plays gram on the young and the restless,['Max Shippee'],Judah Mackey,judah mackey,False,0.000000


#### Overall Exact Match & F1 & Rouge scores

In [ ]:
overall_em = sum(df['em']) / len(df['em'])
overall_f1 = sum(df['f1']) / len(df['f1'])

# Print the overall Exact Match score, F1 and Rouge score
print(f"Overall Exact Match Score: {overall_em * 100:.2f}")
print(f"Overall F1 Score: {overall_f1 * 100:.2f}")
print(f"Rouge1: {rouge_results['rouge1'] * 100:.2f}")
print(f"Rouge2: {rouge_results['rouge2'] * 100:.2f}")
print(f"RougeL: {rouge_results['rougeL'] * 100:.2f}")
print(f"RougeLSum: {rouge_results['rougeLsum'] * 100:.2f}")

Overall Exact Match Score: 34.07
Overall F1 Score: 39.02
Rouge1: 30.99
Rouge2: 18.07
RougeL: 30.82
RougeLSum: 30.80


#### Save with metrics

In [ ]:
eval_with_metrics_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_gpt-3.5-turbo_metrics.csv"

In [ ]:
df.to_csv(eval_with_metrics_file_path, index=False)